In [138]:
import yfinance as yf
import matplotlib.pyplot as plt
import pandas as pd
from datetime import datetime, timedelta

## Define constants to identify assets

In [139]:
# Futures contracts: Update when contracts roll
ES_CONTRACT = 'ESZ23.CME'
US_CONTRACT = 'ZBH24.CBT'
CL_CONTRACT = 'CLF24.NYM'
NG_CONTRACT = 'NGF24.NYM'
GC_CONTRACT = 'GCG24.CMX'
SI_CONTRACT = 'SIH24.CMX'
EC_CONTRACT = '6EZ23.CME'
BP_CONTRACT = '6BZ23.CME'
W_CONTRACT = 'ZWH24.CBT'
C_CONTRACT = 'ZCH24.CBT'
S_CONTRACT = 'ZSF24.CBT'
BTC_CONTRACT = 'BTCZ23.CME'

# Stock tickers: Should seldom or never change
NVDA_TICKER = 'NVDA'

# Column labels for the dataframe
ES_HEADER = 'ESZ23'
US_HEADER = 'USH24'
CL_HEADER = 'CLF24'
NG_HEADER = 'NGF24'
GC_HEADER = 'GCG24'
SI_HEADER = 'SIH24'
EC_HEADER = 'ECZ23'
BP_HEADER = 'BPZ23'
W_HEADER = 'WH24'
C_HEADER = 'CH24'
S_HEADER = 'SF24'
BTC_HEADER = 'BTCZ23'
NVDA_HEADER = 'NVDA'

## Function to retrieve a price series for one asset

In [140]:
def get_price_series(ticker_symbol, label):
    # Get today's and tomorrow's dates
    todays_date = datetime.today().strftime('%Y-%m-%d')
    tomorrows_date = (datetime.today() + timedelta(1)).strftime('%Y-%m-%d')
    # Get prices at 30-minute intervals
    ticker = yf.Ticker(ticker_symbol)
    time_df = ticker.history(start=todays_date, end=tomorrows_date, interval="30m")
    time_series = time_df['Close']
    # Rename the series to indicate the asset
    time_series.rename(label, inplace=True)
    return time_series

## Function to retrieve the open price of the 8:30 Central time bar (stock and agricultural markets open)

In [141]:
def get_open(ticker_symbol):
    # Get today's and tomorrow's dates
    todays_date = datetime.today().strftime('%Y-%m-%d')
    tomorrows_date = (datetime.today() + timedelta(1)).strftime('%Y-%m-%d')
    # Get prices at 30-minute intervals
    ticker = yf.Ticker(ticker_symbol)
    time_df = ticker.history(start=todays_date, end=tomorrows_date, interval="30m")
    # Get the 8:30 Central time open price
    stock_market_open = time_df.at_time("09:30")['Open']
    return stock_market_open

## Get prices for each asset

In [142]:
# Update ticker symbols when contracts roll
es_series = get_price_series(ES_CONTRACT, ES_HEADER)
us_series = get_price_series(US_CONTRACT, US_HEADER)
cl_series = get_price_series(CL_CONTRACT, CL_HEADER)
ng_series = get_price_series(NG_CONTRACT, NG_HEADER)
gc_series = get_price_series(GC_CONTRACT, GC_HEADER)
si_series = get_price_series(SI_CONTRACT, SI_HEADER)
ec_series = get_price_series(EC_CONTRACT, EC_HEADER)
bp_series = get_price_series(BP_CONTRACT, BP_HEADER)
w_series = get_price_series(W_CONTRACT, W_HEADER)
c_series = get_price_series(C_CONTRACT, C_HEADER)
s_series = get_price_series(S_CONTRACT, S_HEADER)
btc_series = get_price_series(BTC_CONTRACT, BTC_HEADER)
nvda_series = get_price_series(NVDA_TICKER, NVDA_HEADER)

In [143]:
# Concatenate all the series into a dataframe
prices = pd.concat([es_series, us_series, cl_series,
                   ng_series, gc_series, si_series,
                    ec_series, bp_series, w_series,
                    c_series, s_series, btc_series,
                    nvda_series], axis="columns")

### For stocks and agricultural markets that open at 8:30 Central time, get the open price of the 8:30 bar and use it as the closing price of the 8:00 bar.

In [146]:
if len(prices) >= 19:
    prices.iloc[18][ES_HEADER] = es_open
    prices.iloc[18][W_HEADER] = w_open
    prices.iloc[18][C_HEADER] = c_open
    prices.iloc[18][S_HEADER] = s_open
    prices.iloc[18][NVDA_HEADER] = nvda_open

C:\Users\steve\AppData\Local\Temp\ipykernel_13632\466120135.py:2: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prices.iloc[18][ES_HEADER] = es_open
C:\Users\steve\AppData\Local\Temp\ipykernel_13632\466120135.py:3: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prices.iloc[18][W_HEADER] = w_open
C:\Users\steve\AppData\Local\Temp\ipykernel_13632\466120135.py:4: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prices.iloc[18][C_HEADER] = c_open
C:\Users\steve\AppData\Local\Temp\ipykernel_13632\466120135.py:5: FutureWarning: Calling float on a single element Series is deprecated and will raise a TypeError in the future. Use float(ser.iloc[0]) instead
  prices.iloc[18][S_HEADER] = s_open
C:\Users\steve

### Convert to Central time and shift times ahead to the ends of the bars 

In [153]:
prices.tz_convert('US/Central').shift(30, freq='T')

,ESZ23,USH24,CLF24,NGF24,GCG24,SIH24,ECZ23,BPZ23,WH24,CH24,SF24,BTCZ23,NVDA
Datetime,,,,,,,,,,,,,
2023-12-05 23:30:00-06:00,4590.25,119.12500,72.320000,2.690,2040.099976,24.615000,1.08015,1.2611,629.25,490.50,1305.75,44185.0,NaN
2023-12-06 00:00:00-06:00,4588.50,119.15625,72.230003,2.689,2042.300049,24.645000,1.08060,1.2612,630.00,490.50,1308.75,44165.0,NaN
2023-12-06 00:30:00-06:00,4587.00,119.21875,72.290001,2.691,2042.800049,24.605000,1.07950,1.2605,629.50,490.75,1309.50,43965.0,NaN
2023-12-06 01:00:00-06:00,4586.25,119.15625,72.370003,2.700,2043.699951,24.615000,1.07930,1.2599,628.75,490.75,1309.50,44180.0,NaN
2023-12-06 01:30:00-06:00,4582.75,119.18750,72.389999,2.695,2045.400024,24.620001,1.07915,1.2597,629.00,490.75,1309.50,44030.0,NaN
2023-12-06 02:00:00-06:00,4586.25,119.37500,72.389999,2.706,2051.500000,24.670000,1.07950,1.2609,630.50,491.00,1309.50,44125.0,NaN
2023-12-06 02:30:00-06:00,4587.50,119.46875,72.320000,2.700,2049.199951,24.615000,1.07850,1.2603,631.75,491.50,1309.50,44240.0,NaN
2023-12-06 03:00:00-06:00,4584.75,119.40625,72.160004,2.695,2042.800049,24.495001,1.07860,1.2594,635.50,492.50,1312.25,44275.0,NaN
2023-12-06 03:30:00-06:00,4584.50,119.37500,72.160004,2.694,2040.300049,24.480000,1.07990,1.2607,636.50,492.00,1311.50,44185.0,NaN


## Get full O-H-L-C data for the S&P 500 contract

In [154]:
es_contract = yf.Ticker(ES_CONTRACT)
todays_date = datetime.today().strftime('%Y-%m-%d')
tomorrows_date = (datetime.today() + timedelta(1)).strftime('%Y-%m-%d')
es_ohlc = es_contract.history(start=todays_date, end=tomorrows_date, interval="30m")

In [155]:
es_ohlc.tz_convert('US/Central')

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Datetime,,,,,,,
2023-12-05 23:00:00-06:00,4588.50,4590.25,4588.25,4590.25,3306,0.0,0.0
2023-12-05 23:30:00-06:00,4590.00,4590.25,4586.75,4588.50,4107,0.0,0.0
2023-12-06 00:00:00-06:00,4588.50,4589.00,4586.75,4587.00,3179,0.0,0.0
2023-12-06 00:30:00-06:00,4586.75,4587.25,4585.75,4586.25,2934,0.0,0.0
2023-12-06 01:00:00-06:00,4586.25,4586.25,4582.50,4582.75,6224,0.0,0.0
2023-12-06 01:30:00-06:00,4582.75,4587.00,4582.50,4586.25,4068,0.0,0.0
2023-12-06 02:00:00-06:00,4586.25,4588.25,4583.75,4587.50,11093,0.0,0.0
2023-12-06 02:30:00-06:00,4587.50,4587.75,4583.25,4584.75,10574,0.0,0.0
2023-12-06 03:00:00-06:00,4584.75,4587.50,4583.25,4584.50,8960,0.0,0.0
